<link href="./table.css", rel="stylesheet">

## 데이터 추출
### 전월세

<table class=simple_table>
    <tr>
        <th>컬럼</td><td>접수년도</td><td>자치구명</td>
        <td>전월세구분</td><td>임대면적</td><td>보증금</td><td>임대료</td>
        <td>건물용도</td><td>신규갱신구분</td>
    </tr>
    <tr>
    <th>데이터 범위</th><td>2023-2024</td>
    </tr>
    <tr>
    <th>데이터크기</th><td>162,215개</td>
    </tr>
</table>

- 컬럼: 접수년도, 자치구명, 전월세구분, 임대면적, 보증금, 임대료, 건물용도, 신규갱신구분
    - 데이터 크기: 162,215 개
    
    

In [62]:
import pandas as pd
pd.options.display.max_columns = 100

# 전월세 csv 파일 불러오기
df_2024 = pd.read_csv('./data/서울특별시_전월세가_2024.csv', encoding='cp949')
df_2023 = pd.read_csv('./data/서울특별시_전월세가_2023.csv')

# 1인가구 면적 필터링
df_2024 = df_2024[df_2024['임대면적(㎡)'] <= 35].reset_index(drop=True).copy()
df_2023 = df_2023[df_2023['임대면적'] <= 35].reset_index(drop=True).copy()

# 임대료 0원 제거
df_2024 = df_2024[df_2024['임대료(만원)'] != 0].reset_index(drop=True).copy()
df_2023 = df_2023[df_2023['임대료(만원)'] != 0].reset_index(drop=True).copy()

# 불필요한 컬럼 제거
df_2024 = df_2024[['접수연도', '자치구명', '전월세 구분', '임대면적(㎡)', '보증금(만원)', '임대료(만원)', '건물용도', '신규갱신여부']].copy()
df_2023 = df_2023[['접수년도', '자치구명', '전월세구분', '임대면적', '보증금(만원)', '임대료(만원)', '건물용도', '신규계약구분']].copy()

# 컬럼명 전체 변경
df_2024.columns = ['접수년도', '자치구명', '전월세구분', '임대면적', '보증금', '임대료', '건물용도', '신규갱신구분']
df_2023.columns = ['접수년도', '자치구명', '전월세구분', '임대면적', '보증금', '임대료', '건물용도', '신규갱신구분']

# 결측값 제거
df_2024 = df_2024.dropna().reset_index(drop=True)
df_2023 = df_2023.dropna().reset_index(drop=True)

# 돈 단위 변경
df_2024['보증금'] = df_2024['보증금'].apply(lambda x: x * 10000)
df_2024['임대료'] = df_2024['임대료'].apply(lambda x: x * 10000)
df_2023['보증금'] = df_2023['보증금'].apply(lambda x: x * 10000)
df_2023['임대료'] = df_2023['임대료'].apply(lambda x: x * 10000)

# 데이터프레임 행으로 합치기
merge_df = pd.concat([df_2023, df_2024])

# index 초기화
merge_df.reset_index(drop=True, inplace=True)

#전월세 산출식: depositMoney + (rentalMoney * 12) / 0.052

merge_df['통합(만원)'] = ((merge_df['보증금'] + (merge_df['임대료'] *12) / 0.052)/10000)

### 서울 각 구별로 산출된 전월세 통합식이 전체 서울 평균과 차이가 있는지
### t-test를 통해서 차이가 있는 지역 표시
- 귀무가설: 차이가 없다.
- 대립가설: 차이가 있다.
- 유의 수준: 0.5

#### 시나리오
- 전체 서울 시 평균과 각 구별 평균의 차이가 있는지 없는지에 대한 귀무가설을 검증
- 검증 데이터 (전체 서울시 데이터, 각 구별 데이터)
- <span style="font-weight:bold;color:red">테스트를 시행할 때마다 p-value의 값이 바뀜</span>


In [63]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import random

merge_df.groupby(by=['자치구명'], as_index=False)[['통합(만원)']].mean()


p_value = {}
t_statistics = {}

#전체데이터
for i in list(merge_df['자치구명'].unique()):
    t, p = sc.stats.ttest_ind(merge_df['통합(만원)'].values,merge_df[merge_df['자치구명']== i]['통합(만원)'].values)
    p_value[i] = p

p_value2 = {}
t_statistics2 = {}

#데이터크기 맞춰줌
all = {}
gu_kind = list(merge_df['자치구명'].unique())
for i in gu_kind:
    num_data = len(merge_df[merge_df['자치구명'] == i]['통합(만원)']) / len(gu_kind)
    all[i] = []
    for j in gu_kind:
        all[i] += list(merge_df[merge_df['자치구명'] == j]['통합(만원)'].values)

for i in gu_kind:
    t, p = sc.stats.ttest_ind(all[i],merge_df[merge_df['자치구명']== i]['통합(만원)'].values)
    p_value2[i] = p
                                                                                                                               
out_name = []
for k,v in p_value2.items():
    if v >= 0.05:
        out_name.append(k)


In [64]:
import folium
import requests
geo_str = requests.get("https://t1.daumcdn.net/cfile/tistory/272C224C58B4BD540B").json()
m = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles="cartodb positron")

json_data = []
for i in geo_str['features']:
    if i['id'] in out_name:
        json_data.append(i)

for i in geo_str['features']:
    adres = i['id'] 
    _icon = folium.DivIcon(html=f"<div style='font-size:10px;font-weight:bold;width:100px'>{adres}</div>")
    if i['id'] in out_name:
        folium.GeoJson(i,style_function=lambda feature: {
                    'fillColor': 'green',  # 채우는 색상
                    'color': 'black',       # 테두리 색상
                    'weight': 2,            # 테두리 두께
                    'dashArray': '5, 5',    # 점선 효과
                    'fillOpacity': 0.5,     # 채우는 색상의 투명도
                }).add_to(m)
    else:
        folium.GeoJson(i,style_function=lambda feature: {
                    'color': 'black',       # 테두리 색상
                    'weight': 2,            # 테두리 두께
                    'dashArray': '5, 5',    # 점선 효과
                    'fillOpacity': 0.5,     # 채우는 색상의 투명도
                }).add_to(m)
    a = list(np.array(i['geometry']['coordinates']).mean(axis=1)[0])
    folium.Marker(location=[a[1],a[0]], icon=_icon).add_to(m)
    

In [65]:
[a[1],a[0]]

[37.5998704500112, 126.98082827261973]

In [66]:
m